In [3]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import json
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'

In [4]:
def get_columns_and_types(df):
    print(f'Types detected: {", ".join([str(i) for i in df.dtypes.unique()])}')
    return {
        'int': [label for label, dtype in df.dtypes.items() if dtype in [int, np.int64]],
        'float': [label for label, dtype in df.dtypes.items() if dtype in [float, np.float64]],
        'object': [label for label, dtype in df.dtypes.items() if dtype in ['O', 'object']],
    }


def convert_dollars_to_float(s, pattern=r"\$|,"):
    """
    convert money-like strings to floats
    """
    if type(s) is str and re.match(pattern, s):
        # if the input is a str containing '$' and/or ',' try to remove those chars and covert the result to a float
        # if this fails, then there is likely text mixed in (like "$195 this week only!")
        try:
            return float(re.sub(pattern, repl='', string=s))
        except:
            return s
    else:
        # otherwise just return the input
        return s


def estimate_y_from_X(data, y_label, X_labels, train_size=0.6, random_state=42, add_constant=True):
    if type(X_labels) not in [list, tuple]:
        X_labels = [X_labels] # in case we pass a scalar
        
    # we know that we have data for 'price' in all observations, but not for 'monthly_price' or 'weekly_price'
    # drop the nans, create a train/test split, build a model and estimate the monthly, then use the model to fill the missing values in the dataset
    reg_data = data[X_labels + [y_label]].dropna()
    y = reg_data[y_label]
    X = reg_data[X_labels]
    if add_constant:
        X = sm.add_constant(reg_data[X_labels])
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)

    print(f'Estimating {y_label}~1+{"+".join(X_labels)}')
    m = sm.OLS(exog=X_train, endog=y_train).fit()
    
    print(f'... Train fit: {r2_score(y_true=y_train, y_pred=m.predict(exog=X_train)):.2f}')
    print(f'... Test fit: {r2_score(y_true=y_test, y_pred=m.predict(exog=X_test)):.2f}')
    
    print(f'... Filling NaNs in {y_label} with estimated data')
    missing = data[y_label][data[y_label].isnull()].index
    data.loc[missing, y_label] = m.predict(exog=sm.add_constant(data[data[y_label].isnull()][X_labels]))
    return data


def get_cleaned_zipcodes(data):
    if 'zipcode_cleaned' in data:
        data = data.drop('zipcode_cleaned', axis=1)

    new_zipcodes = []
    for k, v in data['zipcode'].items():
        if pd.isnull(v) or type(v) is str and len(v) != 5:
            new_zipcodes.append(np.nan)
        else:
            new_zipcodes.append(str(v))
    data = pd.concat([
        data,
        pd.Series(index=data.index, data=new_zipcodes).to_frame('zipcode_cleaned').astype('Int64'),
    ], axis=1)
    return data
    

def classify_y_based_on_X(data, y_label, X_labels, train_size=0.6, random_state=42):
    # loop over existing zip codes and isolate those that are 5 digit strings: we'll consider these to be accurate
    # otherwise, consider them innacurate and replace them with NaN
    # store this as zipcode_clean, for use in the classification algo

    if type(X_labels) is str:
        X_labels = [X_labels]
    
    knn_data = data[[y_label] + X_labels].dropna()
    y = knn_data[y_label]
    X = knn_data[X_labels]    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)
    
    print(f'Estimating {y_label} labels from {"+".join(X_labels)}')
    m_knn = KNeighborsClassifier()
    m_knn.fit(X_train, y_train)
    
    print(f'... Train fit: {m_knn.score(X_train, y_train):.2f}')
    print(f'... Test fit: {m_knn.score(X_test, y_test):.2f}')
    
    print(f'... Filling NaNs in {y_label} with estimated data')
    missing = data[y_label][data[y_label].isnull()].index
    data.loc[missing, y_label] = m_knn.predict(data[data[y_label].isnull()][X_labels])
    return data

# Boston Airbnb Dataset
https://www.kaggle.com/datasets/airbnb/boston?resource=download

Three questions/topics to explore:
- How does **geography** inlfuence Airbnb prices in Boston?
    - What areas/zip codes/neighborhoods are more expensive than others?
- How do **property characteristics** influence Airbnb prices in Boston?
    - Number of bathrooms, bedrooms, beds, square footage, reviews, etc.
- How does **time (seasonality)** influence Airbnb prices in Boston?
    - Period of the week (days), period of the month (weeks), period of the year (months)?
    - We only have about 2 years of data to work with, but we'll see what comes out.

# First, load and clean the data

In [5]:
boston_listings = pd.read_csv('./data/airbnb_boston/listings.csv', index_col='id')
boston_calendar = pd.read_csv('./data/airbnb_boston/calendar.csv')
boston_reviews = pd.read_csv('./data/airbnb_boston/reviews.csv')

boston_listings = boston_listings.map(convert_dollars_to_float)
boston_calendar = boston_calendar.apply(convert_dollars_to_float)
boston_reviews = boston_reviews.apply(convert_dollars_to_float)

# estimate missing monthly_price/weekly_price fields using regression (based on price field)
boston_listings = estimate_y_from_X(data=boston_listings, y_label='monthly_price', X_labels='price')
boston_listings = estimate_y_from_X(data=boston_listings, y_label='weekly_price', X_labels='price')

# estimate missing zip codes using KNN classification (based on latitude and longitude)
boston_listings = classify_y_based_on_X(
    data=get_cleaned_zipcodes(boston_listings), y_label='zipcode_cleaned', X_labels=['latitude', 'longitude'])

Estimating monthly_price~1+price
... Train fit: 0.75
... Test fit: 0.70
... Filling NaNs in monthly_price with estimated data
Estimating weekly_price~1+price
... Train fit: 0.86
... Test fit: 0.82
... Filling NaNs in weekly_price with estimated data
Estimating zipcode_cleaned labels from latitude+longitude
... Train fit: 0.93
... Test fit: 0.90
... Filling NaNs in zipcode_cleaned with estimated data


In [6]:
# identify column types
boston_listing_col_types = get_columns_and_types(boston_listings)
boston_calendar_col_types = get_columns_and_types(boston_calendar)
boston_review_col_types = get_columns_and_types(boston_reviews)
# boston_listing_col_types

Types detected: object, int64, float64, Int64
Types detected: int64, object
Types detected: int64, object


# Add a choropleth (geographical map) and overlay the listings prices from the dataset.
- Thank you for the geojson data: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/boston.geojson?short_path=46589b4
- Use log scale for the prices (so the colorbar isn't too compressed) - or change the scale of the colorbar.

A quick visual inspection of the map allow us to see more expensive listings tend to northward, and in the following neighborhoods (in no particular order):
- West End, North End, South End, Downtown, Leather District, Chinatown, Leather District, South Boston Waterfront, Fenway.

In [7]:
# geoplot of neighborhoods with listings overlaid
with open('./data/airbnb_boston/boston.geojson', 'r') as f:
    geojson = json.load(f)
    
plot_data = boston_listings[['latitude', 'longitude', 'zipcode_cleaned', 'price']].dropna()
plot_data['log_price'] = plot_data['price'].apply(np.log)

fig = px.choropleth(
    data_frame={'name': [i['properties']['name'] for i in geojson['features']]}, 
    geojson=geojson, 
    locations='name', 
    featureidkey="properties.name",
    title='Boston Neighborhoods and Airbnb Prices<br>Colorbar is Log Scale',
)
fig.update_geos(fitbounds="locations", visible=False) 
fig.add_trace(
    px.scatter_geo(
        data_frame=plot_data, 
        lat='latitude', 
        lon='longitude', 
        color='log_price',
        hover_data={
            'latitude': ':.2f', 
            'longitude': ':.2f', 
            'price': ':.2f', 
            'log_price': ':.2f',
        },
    ).data[0])

# relabel the colorbar (as showing log values is confusing)
fig.update_coloraxes(colorbar={
    'title': 'Price',
    'tickvals': plot_data['log_price'].quantile([0.01, 0.999]).values,
    'ticktext': ['Cheaper', 'Pricier'],
})
fig.update_layout(showlegend=False)

fig.show()

In [8]:
pd.concat([
    boston_listings[['price', 'neighbourhood_cleansed']].groupby('neighbourhood_cleansed').mean().squeeze().to_frame('Mean Price'),
    boston_listings[['price', 'neighbourhood_cleansed']].groupby('neighbourhood_cleansed').median().squeeze().to_frame('Median Price'),
], axis=1).sort_values(by='Median Price').style.format('{:.2f}').set_caption('Mean and Median Price by Neighborhood<br>Sorted by Median')

,Mean Price,Median Price
neighbourhood_cleansed,,
Hyde Park,86.55,58.00
Mattapan,75.12,67.00
Dorchester,91.64,72.00
Roslindale,98.43,76.50
Longwood Medical Area,138.44,84.00
Allston,112.31,85.00
West Roxbury,107.11,85.00
Mission Hill,121.97,89.50
Brighton,118.77,90.00


In [9]:
boston_listings[boston_listing_col_types['float']].sort_index().dropna(how='all', axis=1)

,latitude,longitude,bathrooms,bedrooms,beds,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,
3353,42.355017,-71.127590,1.0,1.0,1.0,NaN,40.0,400.000000,1200.000000,100.0,50.0,0.0,90.0,9.0,8.0,9.0,9.0,9.0,9.0,0.38
5506,42.329809,-71.095595,1.0,1.0,1.0,225.0,145.0,980.000000,3000.000000,NaN,40.0,0.0,91.0,9.0,10.0,10.0,10.0,9.0,10.0,0.40
6695,42.329941,-71.093505,1.0,1.0,2.0,NaN,195.0,1175.968465,4248.743174,NaN,70.0,8.0,95.0,9.0,10.0,10.0,10.0,8.0,9.0,0.54
6976,42.292438,-71.135765,1.0,1.0,1.0,NaN,65.0,395.000000,1350.000000,NaN,NaN,20.0,98.0,10.0,9.0,10.0,10.0,9.0,10.0,0.47
8792,42.358497,-71.062011,1.0,1.0,1.0,NaN,154.0,750.000000,2900.000000,1000.0,250.0,0.0,91.0,10.0,9.0,10.0,9.0,10.0,10.0,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14924831,42.381921,-71.068837,1.0,1.0,1.0,NaN,190.0,1147.954778,4140.063870,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14928000,42.300476,-71.076970,1.0,1.0,1.0,NaN,55.0,391.585233,1205.722666,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14928333,42.337349,-71.048150,1.0,1.0,1.0,NaN,90.0,587.681041,1966.477793,NaN,25.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sns.scatterplot(
    data=boston_listings[['price', 'square_feet']].dropna(),
    x='square_feet', y='price')

In [ ]:
sns.pairplot(
    boston_listings[boston_listing_col_types['float']].sort_index().dropna(how='all', axis=1) \
        .drop(['latitude', 'longitude', 'weekly_price', 'monthly_price'], axis=1)
)

In [ ]:
boston_calendar[boston_calendar.price.isnull()]

In [ ]:
boston_calendar[boston_calendar.price.notnull()]

In [ ]:
# convert price column to a float, assuming format of $#.#
boston_calendar.price = [float(i.replace('$', '').replace(',', '')) if type(i) is str else i for i in boston_calendar.price]

# concert date to a datetime object
boston_calendar.date = [pd.Timestamp(i) if type(i) is str else i for i in boston_calendar.date]

# add a month, year columns for seasonality analysis
boston_calendar['month'] = [i.month for i in boston_calendar.date]
boston_calendar['year'] = [i.year for i in boston_calendar.date]

In [ ]:
sns.boxplot(
    data=boston_calendar.dropna(),
    x='month',
    y='price',
)